# Combining original and flipped segmentations to get purely right-ventricle segmentations

In [ ]:
from pathlib import Path
import sys
import math
from typing import List, Tuple

import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from dotenv import dotenv_values

import echonet

config = dotenv_values(".env")

# Just some types for us to use in type hints to make dev easier
Point = List[np.intp]
Box = Tuple[Point, Point, Point, Point]
Rectangle = Tuple[Point, Tuple[float, float], float] # [centre, (width, height), angle]

# Directories containing both original and flipped segmentation masks
LEFT_SEGMENTATION_DIR = Path(config["LEFT_SEGMENT_DIR"]) # e.g. LEFT_SEGMENT_DIR="output/segmentation/all-patients"
LEFT_SEGMENTATION_MASK_DIR = LEFT_SEGMENTATION_DIR / "segmentation_masks" # remember that you have to have run the modified segmentation.py to get this segmentations sub-directory!
FLIPPED_SEGMENTATION_DIR = Path(config["FLIPPED_SEGMENT_DIR"]) # e.g. FLIPPED_SEGMENT_DIR="output/segmentation/flipped"
FLIPPED_SEGMENTATION_MASK_DIR = FLIPPED_SEGMENTATION_DIR / "segmentation_masks"

ECHONET_VIDEO_DIR = Path(config["ECHONET_VIDEO_DIR"]) # e.g. ECHONET_VIDEO_DIR="/home/lex/data/echonet-data/Videos"

# Can assign these colours to numpy arrays so long as the colours are stored in
# the last axis of the target array (e.g. image.shape =(112, 112, 3), but not
# image.shape = (3, 112, 112)). 
# Just do image[y_vals, x_vals] = MAGENTA
# IMPORTANT: if using within an *opencv* function, you'll want to do 
# COLOUR.tolist() to convert these to python primitives, else opencv complains about
# datatypes
RED = np.array([255, 0, 0])
GREEN = np.array([0, 255, 0])
BLUE = np.array([0, 0, 255])
ORANGE = np.array([255, 165, 0])
LIGHT_GREY = np.array([211, 211, 211])
MAGENTA = np.array([255, 0, 255])
YELLOW = np.array([255, 255, 0])

In [ ]:
def get_heights(mask: np.ndarray) -> List[int]:
    """Returns array of heights of mask for each frame!"""
    frame_indices, row_indices = np.where(mask.any(axis=1)==True)

    heights = []
    for frame_index in range(mask.shape[0]):
        this_frame = frame_indices == frame_index

        if len(row_indices[this_frame]) == 0:
            # print(f"Frame #{frame_index} appears to have no segmentations? Treating this as zero height...")
            heights.append(0)
            continue

        min_row, max_row = (min(row_indices[this_frame]), max(row_indices[this_frame]))
        height = max_row - min_row
        heights.append(height)

    heights = np.array(heights)
    return heights

# Filtering out garbage data
Here, we'll be trying to discard any videos that are not A4C views.

In [ ]:
def get_angle(rect: Rectangle) -> float:
    _, (width, height), angle = rect
    if width < height:
        return 0 - angle
    elif height <= width:
        return 90 - angle

In [ ]:
BOTTOM_RIGHT = 0
BOTTOM_LEFT = 1
TOP_RIGHT = 2
TOP_LEFT = 3

def find_corner(rect: Rectangle, which: int) -> Point:
    """
    (Attempts to) find the "bottom-right" corner of the given rectangle. This 
    can at least handle rectangles rotated by an angle in the range [0, 90).
    """
    # angle = rect[-1]
    angle = get_angle(rect)
    box = np.intp(cv2.boxPoints(rect))

    # Critical angle regions are: [-90, -45), [-45, 0), [0, 45), [45, 90] ?
    if (-90 <= angle < -45) or (0 <= angle < 45):
        if which == BOTTOM_LEFT:
            return box[3]
        elif which == TOP_LEFT:
            return box[0]
        elif which == TOP_RIGHT:
            return box[1]
        elif which == BOTTOM_RIGHT:
            return box[2]
    elif (-45 <= angle < 0) or (45 <= angle <= 90):
        if which == BOTTOM_LEFT:
            return box[0]
        elif which == TOP_LEFT:
            return box[1]
        elif which == TOP_RIGHT:
            return box[2]
        elif which == BOTTOM_RIGHT:
            return box[3]

In [ ]:
def mask_to_image(mask: np.ndarray, max_val: int = 255) -> np.ndarray:
    """
    Converts a boolean mask array to a pure black and white image. Useful if
    you start with a mask but then want to find contours and perform other image
    analysis on that mask    
    """
    return mask.astype(np.uint8) * max_val

def image_to_mask(image: np.ndarray, threshold: int = 1) -> np.ndarray:
    """
    Converts a black and white image to a boolean mask, selecting every pixel 
    whose intensity is **greater than or equal to the threshold**
    """
    return image >= threshold

In [ ]:
# Reload here so we can just change the VIDEONAME in .env without having to 
# go back and rerun top cell
config = dotenv_values(".env")
VIDEONAME = config["VIDEONAME"] # e.g. VIDEONAME="0X1DB488AC3583E3D6"

LEFT_SEGMENTATION_MASK_FP = LEFT_SEGMENTATION_MASK_DIR / f"{VIDEONAME}.npy"
FLIPPED_SEGMENTATION_MASK_FP = FLIPPED_SEGMENTATION_MASK_DIR / f"{VIDEONAME}.npy"
ECHONET_VIDEO_FP = ECHONET_VIDEO_DIR / f"{VIDEONAME}.avi"

left_segmentation_masks = np.load(LEFT_SEGMENTATION_MASK_FP)
flipped_segmentation_masks = np.load(FLIPPED_SEGMENTATION_MASK_FP)
flipped_segmentation_masks = np.flip(flipped_segmentation_masks, -1) # Flip back to original orientation

# Subtract those mistaken left ventricle parts from the right segmentation
right_segmentation_masks = flipped_segmentation_masks & ~left_segmentation_masks

left_segmentations = mask_to_image(left_segmentation_masks)
right_segmentations = mask_to_image(right_segmentation_masks)

echonet_video = echonet.utils.loadvideo(str(ECHONET_VIDEO_FP))
echonet_video = echonet_video.transpose((1, 2, 3, 0))

num_frames, height, width, num_channels = echonet_video.shape

In [ ]:
def make_line(p1: Point, p2: Point, xs: List[int]) -> List[Point]:
    """
    Extends a line between the given pair of points for all given x values and
    returns the list of points on this line.
    
    This clips the points for you in case any of the given x values correspond to
    y values outside of the image's borders.
    """
    x1, y1 = p1
    x2, y2 = p2
    
    gradient = (y2 - y1) / (x2 - x1)
    ys = gradient * (xs - x1) + y1
    ys = np.intp(ys)
    
    all_points = list(zip(xs, ys))
    points_within_frame = []
    # Only include points that would be within the image's borders
    for point in all_points:
        x, y = point
        if (0 <= x < width) and (0 <= y < height):
            points_within_frame.append(point)

    return points_within_frame

In [ ]:
def get_min_area_rect(image: np.ndarray) -> Rectangle:
    """
    Finds the largest contour in the given image, and returns the minimum area
    rectangle that bounds it.

    Returns
    -------
    ((centre_x, centre_y), (width, height), angle)
    """
    contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Assume we're only interested in max area contour
    areas = [cv2.contourArea(cnt) for cnt in contours]
    max_index = np.argmax(areas)
    biggest_contour = contours[max_index]

    min_area_rect = cv2.minAreaRect(biggest_contour)
    return min_area_rect

In [ ]:
def get_min_area_box(image: np.ndarray) -> Box:
    """
    Similar to get_min_area_rect(), but instead returns the coordinates of the
    box's four corners.
    """
    min_area_rect = get_min_area_rect(image)
    box = np.intp(cv2.boxPoints(min_area_rect))
    
    return box

In [ ]:
def perpendicular_distance_to_line(line: List[List[int]], point: List[int]) -> float:
    """
    Returns the perpendicular distance from a straight line to a given point.

    Parameters
    ----------
    line: List[List[int]]
        `line` is a *p0 - angleair* of points, where each point contains a single x and y
        value.
    point: List[int]
        A pair of x and y values.
    """
    (x1, y1), (x2, y2) = line
    x3, y3 = point

    m = (y2 -y1) / (x2 - x1)
    a = -1
    b = 1 / m
    c = x1 - y1 / m

    d = abs(a * x3 + b * y3 + c) / math.sqrt(a**2 + b**2)
    return d

In [ ]:
# Define LV boxes and rectangles
LV_boxes = [get_min_area_box(left_segmentation) for left_segmentation in left_segmentations]
LV_rects = [get_min_area_rect(left_segmentation) for left_segmentation in left_segmentations]
LV_lines = [[find_corner(rect, BOTTOM_LEFT), find_corner(rect, TOP_LEFT)] for rect in LV_rects]
LV_angles = [get_angle(rect) for rect in LV_rects]

plt.hist(LV_angles, bins=20)

In [ ]:
# Only include largest contour of right segmentation for each frame. This removes
# any smaller, disconnected blobs
right_segmentations_copy = right_segmentations.copy()
right_segmentations = np.zeros(right_segmentations.shape, dtype=right_segmentations.dtype)
for i, right_segmentation in enumerate(right_segmentations_copy):
    right_contours, hierarchy = cv2.findContours(right_segmentation, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(cnt) for cnt in right_contours]
    max_index = np.argmax(areas)
    biggest_contour = right_contours[max_index]

    biggest_right_segmentation = np.zeros(right_segmentation.shape)
    biggest_right_segmentation = cv2.drawContours(biggest_right_segmentation, [biggest_contour], -1, 255, -1)
    right_segmentations[i] = biggest_right_segmentation

# Update masks too, would be nicer to have these masks and images automatically linked though!
right_segmentation_masks = image_to_mask(right_segmentations)

In [ ]:
# Trim RV segmentations based on LV bounding box
right_segmentations = np.zeros(shape=right_segmentations.shape, dtype=right_segmentations.dtype)

bottom_cutoff_points_list = []
top_cutoff_points_list = []
right_cutoff_points_list = []

cutoff_x = np.intp(np.arange(0, width))

for i, (LV_box, LV_rect, right_segmentation_mask) in enumerate(zip(LV_boxes, LV_rects, right_segmentation_masks)):
    if i == 10:
        print("We're here")

    top_left_LV = find_corner(LV_rect, TOP_LEFT)
    top_right_LV = find_corner(LV_rect, TOP_RIGHT)
    bottom_left_LV = find_corner(LV_rect, BOTTOM_LEFT)
    bottom_right_LV = find_corner(LV_rect, BOTTOM_RIGHT)

    bottom_cutoff_points = make_line(bottom_left_LV, bottom_right_LV, cutoff_x)
    bottom_cutoff_points_list.append(bottom_cutoff_points)

    top_cutoff_points = make_line(top_left_LV, top_right_LV, cutoff_x)
    top_cutoff_points_list.append(top_cutoff_points)

    right_cutoff_points = make_line(bottom_left_LV, top_left_LV, cutoff_x)
    right_cutoff_points_list.append(right_cutoff_points)

    # Note we update the masks here, so we don't need to call image_to_mask()
    # at end of this for loop!
    for x, y in bottom_cutoff_points:
        right_segmentation_mask[y:, x] = False
    for x, y in right_cutoff_points:
        right_segmentation_mask[:y, x] = False

    right_segmentations[i] = mask_to_image(right_segmentation_mask)


In [ ]:
# Calculate RV areas
# Since we have already eliminated all but the largest contour, we simply always
# use hardcoded index of 0 to access it!
# Then extra index zero for contours because of opencv's return values
RV_contours_list = [cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)[0][0] for frame in right_segmentations]
RV_areas = [cv2.contourArea(contour) for contour in RV_contours_list]

In [ ]:
RV_contours_list = [cv2.findContours(frame, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) for frame in right_segmentations]

In [ ]:
contours_list = [contours[0] for contours in RV_contours_list]

In [ ]:
num_contours = [len(cnt) for cnt in contours_list]

In [ ]:
num_contours

In [ ]:
# Calculate angle for bounding box of LV in each frame
LV_box_angles = np.zeros(num_frames)
for i, left_segmentation in enumerate(left_segmentations):
    *_, angle = get_min_area_rect(left_segmentation)
    LV_box_angles[i] = angle

print(f"Mean LV box angle: {np.mean(LV_box_angles)} +/- {np.std(LV_box_angles)}")

# Check how many times the box goes beyond three standard deviations
mean, std = np.mean(LV_box_angles), np.std(LV_box_angles)
bad_angle_mask = (LV_box_angles < mean - 1 * std) | ((LV_box_angles > mean + 1 * std))
num_bad_angles = bad_angle_mask.sum()
print(f"Found {num_bad_angles} frames with suspicious LV bounding box angles")

In [ ]:
# Find septum widths
septum_widths = []

for right_segmentation, LV_line in zip(right_segmentations, LV_lines):
    RV_rect = get_min_area_rect(right_segmentation)
    RV_bottom_right = find_corner(RV_rect, BOTTOM_RIGHT)

    x3, y3 = RV_bottom_right
    # frame[y3:y3+2, x3:x3+2] = ORANGE
    septum_width = perpendicular_distance_to_line(LV_line, RV_bottom_right)
    septum_widths.append(septum_width)
    # print(f"Septum: {septum_width}")

print(f"Mean septum width: {np.mean(septum_widths)} +/- {np.std(septum_widths)}")

In [ ]:
import matplotlib.pyplot as plt

plt.hist(septum_widths, bins=10)

In [ ]:
RV_boxes = [get_min_area_box(right_segmentation) for right_segmentation in right_segmentations]

In [ ]:
# Use average estimated septum width and translate the LV segmentation's inner
# edge by that amount to guess the right edge of the RV.
mean_septum_width = np.mean(septum_widths)

RV_boxes = []
RV_lines = []
for LV_box, LV_rect, LV_line, right_segmentation_mask, right_segmentation in zip(LV_boxes, LV_rects, LV_lines, right_segmentation_masks, right_segmentations):
    LV_angle = LV_rect[-1]
    
    # Since we translate points *left*, and not necessarily perpendicular to LV
    # line, we include factor of sin(angle)
    translate_x = mean_septum_width / math.sin(math.radians(LV_angle))
    RV_line = [point.copy() for point in LV_line]
    for point in RV_line:
        point[0] -= translate_x

    RV_lines.append(RV_line)

    RV_box = get_min_area_box(right_segmentation)
    # Just in case we need to access this later
    RV_boxes.append(RV_box)

    # Now remove any pixels in RV segmentation that go beyond its expected inner
    # edge
    # Note this is a greedier "right cutoff" than before, should probs just name this better!
    right_cutoff_points = make_line(RV_line[0], RV_line[1], cutoff_x)
    for x, y in right_cutoff_points:
        right_segmentation_mask[:y, x] = False

# Update RV segmentation *images* based on these new masks
right_segmentations = mask_to_image(right_segmentation_masks)

In [ ]:
WINDOW = "Mask"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

i = 0
is_playing = True


while True:
    if i >= num_frames:
        i = 0
        print("Video looped!")
    elif i < 0:
        i = num_frames - 1

    # Copy data for this particular frame
    frame = echonet_video[i].copy()
    left_segmentation_mask = left_segmentation_masks[i].copy()
    left_segmentation = left_segmentations[i].copy()
    right_segmentation_mask = right_segmentation_masks[i].copy()
    right_segmentation = right_segmentations[i].copy()

    # this_area = RV_areas[i]
    # LV_line = LV_lines[i]
    # RV_line = RV_lines[i]
    LV_box = LV_boxes[i]
    LV_rect = LV_rects[i]
    LV_angle = get_angle(LV_rect)
    # print(f"{LV_angle:.2f}: {LV_box}")
    # RV_box = RV_boxes[i]
    RV_box = get_min_area_box(right_segmentation)
    RV_rect = get_min_area_rect(right_segmentation)
    RV_angle = get_angle(RV_rect)
    print(f"{RV_angle:.2f}: {RV_box}: {RV_rect}")

    # 
    # if i > 0: 
    #     prev_area = RV_areas[i - 1]
    #     growth = this_area / prev_area
        # print(f"RV Area growth: {growth:.2f}")
        # if growth > 1.2:
        #     frame[:10, :] = ORANGE

    # Add any drawings to the ultrasound here
    # frame[left_segmentation_mask] = RED
    frame[right_segmentation_mask] = BLUE
    # Note this draws the outdated RV_box, before we cut out the septum!
    cv2.drawContours(frame,[RV_box],0, YELLOW.tolist())
    # cv2.drawContours(frame,[LV_box],0, YELLOW.tolist())
    # cv2.line(frame, np.intp(LV_line[0]), np.intp(LV_line[1]), ORANGE.tolist())
    # cv2.line(frame, np.intp(RV_line[0]), np.intp(RV_line[1]), RED.tolist())

    # RV_rect = RV_rects[i]
    bl, tl, br, tr = find_corner(RV_rect, BOTTOM_LEFT), find_corner(RV_rect, TOP_LEFT), find_corner(RV_rect, BOTTOM_RIGHT), find_corner(RV_rect, TOP_RIGHT)
    frame[br[1], br[0]] = RED

    top_border = np.zeros((height // 8, width, 3), dtype=frame.dtype)
    top_border[:, :] = np.expand_dims(LIGHT_GREY, (0, 1))
    cv2.putText(top_border, f"Frame {i+1}/{num_frames}", org=(5,10), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.25, color=(255, 0, 0))
    frame = np.concatenate([top_border, frame], axis=0)
    frame = np.flip(frame, -1)
    cv2.imshow(WINDOW, frame)

    keypress = cv2.waitKey(50) & 0xFF
    if keypress == ord('q'):
        break
    elif keypress == ord(' '):
        is_playing = not is_playing
    elif keypress == ord('a'):
        i -= 1
    elif keypress == ord('d'):
        i += 1
    else:
        if is_playing:
            i += 1

cv2.destroyAllWindows()

In [ ]:
LV_rects[9]

In [ ]:
print(LV_rects[20])
print(LV_rects[21])
print(LV_rects[24])

In [ ]:
print(f"{LV_rects[2][-1]:.256f}")

In [ ]:
# This cell is just for playing around with rotated rectangles and their corners,
# while we figure out how on earth opencv determines the angle in its return value
# for cv2.minAreaRect()!
WINDOW = "Mask"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)

frame = np.zeros((112, 112), dtype=np.uint8)
rows,cols = frame.shape
width = 30
height = 60
angle = 135
centre_x = cols // 2
centre_y = rows // 2
corner1 = (centre_x - width // 2, centre_y - height // 2)
corner2 = (centre_x + width // 2, centre_y + height // 2)
frame = cv2.rectangle(frame, corner1, corner2, 255, -1)

matrix = cv2.getRotationMatrix2D((56, 56), angle, 1)
frame = cv2.warpAffine(frame, matrix, (cols, rows))

box = get_min_area_box(frame)
rect = get_min_area_rect(frame)
our_angle = get_angle(rect)
print(our_angle)
print(rect)
print(box)
print("HERE?")
frame = np.zeros((112, 112), dtype=np.uint8)
cv2.drawContours(frame,[box],0, 255)

# bottom_left = find_corner(rect, BOTTOM_LEFT)
# frame[bottom_left[1], bottom_left[0]] = 255

frame = np.flip(frame, axis=-1)

while True:
    cv2.imshow(WINDOW, frame)

    keypress = cv2.waitKey(50) & 0xFF
    if keypress == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
get_min_area_box(frame)